In [2]:
%%javascript
require(["codemirror/keymap/sublime", "notebook/js/cell", "base/js/namespace"],
    function(sublime_keymap, cell, IPython) {
        cell.Cell.options_default.cm_config.keyMap = 'sublime';
    
        var cells = IPython.notebook.get_cells();
        for(var cl=0; cl< cells.length ; cl++){
            cells[cl].code_mirror.setOption('keyMap', 'sublime');
        }
    }
);

<IPython.core.display.Javascript object>

In [3]:
# change the cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 90% !important; }</style>"))

In [6]:
%load_ext autoreload
%autoreload 2
import os
import cooler
import cooltools
import numpy as np
from cooltools.api import eigdecomp
import bioframe
from pathlib import Path
import multiprocess as mp

from copy import copy

%matplotlib inline
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### some of the input cooler files that we used to combine/recombine/subset etc

In [7]:
ins_samples = {
    "Hap1-WT-combined.hg19" : "Hap1-WT-combined.mcool",
    "PolII-IAA.hg19" : "PolII-IAA.hg19.mapq_30.1000.mcool",
    "PolII-NT.hg19" : "PolII-NT.hg19.mapq_30.1000.mcool",
    "all_CTCF_combined.hg19" : "all_combined.mcool",
    "noCTCF_IAA_combined.hg19" : "noCTCF_IAA_combined.mcool",
    
    "all_CTCF_combined.hg19" : "all_combined.mcool",
    "noCTCF_IAA_combined.hg19" : "noCTCF_IAA_combined.mcool",
    
    "CkoCT442_NT_pool.hg19" : "CkoCT442-NT-pool.mcool",
    "CkoCT442_IAA_pool.hg19" : "CkoCT442-IAA-pool.mcool",
    # RAD21 degron for HCT from Rao et al 2017 ...
    "RAD21-IAA.hg19" : "RAD21-AID-IAA-6H.hg19.mapq_30.1000.mcool",
    "RAD21-NT.hg19" : "RAD21-AID-NT.hg19.mapq_30.1000.mcool",

    # WAPL KO for HAP1 from Haarhuis 2017 ...
    "WAPL-KO1.hg19" : "Haarthuis-WAPL-KO1.14.hg19.mapq_30.1000.mcool",
    "WAPL-KO3.hg19" : "Haarthuis-WAPL-KO3.3.hg19.mapq_30.1000.mcool",
    "WAPL-WT.hg19" : "Haarthuis-HAP1-WT.hg19.mapq_30.1000.mcool",

    "CkoC442-IAA48H-R1-T1.hg19" : "CkoC442-IAA48H-R1-T1__hg19.hg19.mapq_30.1000.mcool",
    "CkoC442-NT-R1-T1.hg19" : "./wt_like_coolers_to_combine/CkoC442-NT-R1-T1__hg19.hg19.mapq_30.1000.mcool",
    "CkoCT442-AAVS1sg2-4-NT-R1-T1.hg19" : "CkoCT442-AAVS1sg2-4-NT-R1-T1__hg19.hg19.mapq_30.1000.mcool",
    "CkoCT442-DDX55sg2-B-NT-R1-T1.hg19" : "CkoCT442-DDX55sg2-B-NT-R1-T1__hg19.hg19.mapq_30.1000.mcool",
    "G1-NT.hg19" : "G1-NT.hg19.1kb.mcool",
    "G1-IAA.hg19" : "G1-IAA.hg19.1kb.mcool",
    
    # Mutants time !!!
    # Knock-outs - genes were deleted ...
    "mutControl-NT.hg19":"CkoCT442-AAVS1-NT-pool.mcool",
    "mutControl-IAA.hg19":"CkoCT442-AAVS1-IAA-pool.mcool",
    "mutDDX55-NT.hg19":"DDX55-clones-NT.hg19.mapq_30.1000.mcool",
    "mutDDX55-IAA.hg19":"DDX55-clones-IAA.hg19.mapq_30.1000.mcool",
    "mutTAF5L-NT.hg19":"TAF5L-clones-NT.hg19.mapq_30.1000.mcool",
    "mutTAF5L-IAA.hg19":"TAF5L-clones-IAA.hg19.mapq_30.1000.mcool",
    # Knock-downs - genes were silenced/deplted (siRNA) ...
    "siControl-NT.hg19":"siCTRL-NT.hg19.mapq_30.1000.mcool",
    "siControl-IAA.hg19":"siCTRL-IAA.hg19.mapq_30.1000.mcool",
    "siDDX55-NT.hg19":"siDDX55-NT.hg19.mapq_30.1000.mcool",
    "siDDX55-IAA.hg19":"siDDX55-IAA.hg19.mapq_30.1000.mcool",
    "siTAF5L-NT.hg19":"siTAF5L-NT.hg19.mapq_30.1000.mcool",
    "siTAF5L-IAA.hg19":"siTAF5L-IAA.hg19.mapq_30.1000.mcool",
    
    # PlaB (splicing inhibition) two replicates pooled together - known to affect R-loops (mostly destabilize)
    "CtrlPlaB-NT.hg19" : "NT-hg19-combined-90000000.mcool",
    "CtrlPlaB-IAA.hg19" : "IAA-hg19-combined-90000000.mcool",
    "PlaB-NT.hg19" : "NT-PlaB-hg19-combined-90000000.mcool",
    "PlaB-IAA.hg19" : "IAA-PlaB-hg19-combined-90000000.mcool",
    
    ## Pool HAP1 WT, NO TIR1 and TIR1
    # this is a special subsampling case for the standard CTCF-degron(+aux)
    # but normalized, such that the number of reads, matches the NO-TIR1
    "Ctrl500M-noTIR1.hg19" : "CkoC44-NO-TIR1-pool.mcool",
    "Ctrl500M-wtHAP1.hg19" : "Hap1-WT-combined-500000000.mcool",
    "Ctrl500M-CT442-NT.hg19" : "CkoCT442-NT-pool-500000000.mcool",
    "Ctrl500M-CT442-IAA.hg19" : "CkoCT442-IAA-pool-500000000.mcool",
    
    ## A group for comparing PolII degron samples - i.e. with matched read depth to Hap1WT
    # "Hap1-WT-combined.hg19" : "Hap1-WT-combined.mcool",
    "PolII-800M-IAA.hg19" : "PTB2539-IAA-pool-800000000.mcool",
    "PolII-800M-NT.hg19" : "PTB2539-NT-pool-800000000.mcool",
}


In [8]:
# %%bash
# cname="./CkoCT442-NT-pool.mcool"
# for uri in $(cooler ls $cname);
# do
#     echo $uri
#     cooler balance --check $uri
# done

In [11]:
binsize = 5_000
binsize_human = f"{int(binsize/1_000)}kb"
diamond = 100_000
diamond_human = f"{int(diamond/1_000)}kb"

hg19_chromsizes = bioframe.fetch_chromsizes("hg19")

# iterate over samples to calculate insulation on:
for k,cname in ins_samples.items():
    target_bw_file = Path(f"{k}.{binsize_human}.{diamond_human}.bw")
    if target_bw_file.is_file():
        print("already exist !")
        print(target_bw_file)
        continue
    else:
        print("working on ...")
        print(k,cname)
        ######################################
        ! cooltools insulation \
            -o {k}.{binsize_human}.{diamond_human}.tsv \
            {cname}::/resolutions/{binsize} \
            {diamond}
        ###################################### 
        df = bioframe.read_table(
                f"{k}.{binsize_human}.{diamond_human}.tsv",
                header=0
        )
        ######################################
        bioframe.to_bigwig(
            df,
            hg19_chromsizes,
            f"{k}.{binsize_human}.{diamond_human}.bw",
            value_field = f"log2_insulation_score_{diamond}"
        )


already exist !
Hap1-WT-combined.hg19.5kb.100kb.bw
already exist !
PolII-IAA.hg19.5kb.100kb.bw
already exist !
PolII-NT.hg19.5kb.100kb.bw
already exist !
all_CTCF_combined.hg19.5kb.100kb.bw
already exist !
noCTCF_IAA_combined.hg19.5kb.100kb.bw
already exist !
CkoCT442_NT_pool.hg19.5kb.100kb.bw
already exist !
CkoCT442_IAA_pool.hg19.5kb.100kb.bw
already exist !
RAD21-IAA.hg19.5kb.100kb.bw
already exist !
RAD21-NT.hg19.5kb.100kb.bw
already exist !
WAPL-KO1.hg19.5kb.100kb.bw
already exist !
WAPL-KO3.hg19.5kb.100kb.bw
already exist !
WAPL-WT.hg19.5kb.100kb.bw
already exist !
CkoC442-IAA48H-R1-T1.hg19.5kb.100kb.bw
already exist !
CkoC442-NT-R1-T1.hg19.5kb.100kb.bw
already exist !
CkoCT442-AAVS1sg2-4-NT-R1-T1.hg19.5kb.100kb.bw
already exist !
CkoCT442-DDX55sg2-B-NT-R1-T1.hg19.5kb.100kb.bw
already exist !
G1-NT.hg19.5kb.100kb.bw
already exist !
G1-IAA.hg19.5kb.100kb.bw
already exist !
mutControl-NT.hg19.5kb.100kb.bw
already exist !
mutControl-IAA.hg19.5kb.100kb.bw
already exist !
mutDDX55-NT.

In [8]:
# # checking sequencing depth of several samples ...
# !cooler info all_combined.mcool::/resolutions/25000 | grep sum
# !cooler info all_combined.mcool::/resolutions/25000 | grep nnz
# !cooler info noCTCF_IAA_combined.mcool::/resolutions/25000 | grep sum
# !cooler info noCTCF_IAA_combined.mcool::/resolutions/25000 | grep nnz

## Calculating compartment status per arm for several samples ...

In [9]:
def rename_arms(name):
    """
    turn UCSC genomic region name
    into a human readable name of chrom arm
    """
    chrom,coords = name.split(":")
    start,end = coords.split("-")
    if start == "0":
        return chrom+"p"
    else:
        return chrom+"q"

hg19_chromsizes = bioframe.fetch_chromsizes('hg19', as_bed=True)
hg19_cens = bioframe.fetch_centromeres('hg19')
hg19_arms = bioframe.split(hg19_chromsizes, hg19_cens, cols_points=['chrom', 'mid'])
# # Select only chromosomes that are present in the cooler. 
# # This step is typically not required! we call it only because the test data are reduced. 
# hg19_chromsizes = hg19_chromsizes.set_index("chrom").loc[hg19_chromsizes.chrom].reset_index() 
hg19_arms = hg19_arms.set_index("chrom").loc[hg19_chromsizes.chrom.values[:-1]].reset_index()
# call this to automaticly assign names to chromosomal arms:
hg19_arms = bioframe.parse_regions(hg19_arms)
hg19_arms["name"] = hg19_arms["name"].apply(rename_arms)
hg19_arms.head()
hg19_arms.to_csv("hg19_arms.bed",sep="\t",header=False,index=False)

In [10]:
# get some cooler to extract bin table - for reference track extraction ...
binsize = 25_000
human_binsize = f"{int(binsize/1_000)}kb"
db = "hg19"
# input samples here
# make sure we do not regenerate the same reference track again ...
target_ref_track = Path(f"frac_gene_coverage.{human_binsize}.bedGraph")
if target_ref_track.is_file():
    print("already exist !")
    print(target_ref_track)
else:
    print(f"working on ... {human_binsize} gene coverage for {db} ...")
    ######################################
    # extract gene coverage for a given bin-size ...
    # we need SOME cooler to get a bin table with that binsize ...
    s= 'CkoCT442_NT_pool.hg19'
    fname = ins_samples[s]
    clr = cooler.Cooler(f"{fname}::/resolutions/{binsize}")
    # gene cov extraction !
    gene_cov_df = bioframe.frac_gene_coverage(
                clr.bins()[:],
                db,
                )
    # save coverage track ...
    gene_cov_df[["chrom","start","end","coverage"]] \
        .to_csv(
            target_ref_track,
            sep="\t",
            header=False,
            index=False
    )


already exist !
frac_gene_coverage.25kb.bedGraph


In [11]:
binsize = 25_000
human_binsize = f"{int(binsize/1_000)}kb"
# iterate over samples to calculate insulation on:
for k,cname in ins_samples.items():
    target_bw_file = Path(f"{k}.EV.{human_binsize}.cis.bw")
    if target_bw_file.is_file():
        print("already exist !")
        print(target_bw_file)
        continue
    else:
        print("working on ...")
        print(k, cname)
        print(f"to generate {target_bw_file} ...")
        ######################################
        ! cooltools call-compartments \
          --reference-track frac_gene_coverage.{human_binsize}.bedGraph \
          --regions hg19_arms.bed \
          --contact-type cis \
          --n-eigs 3 \
          -v \
          -o {k}.EV.{human_binsize} \
          --bigwig \
        {cname}::/resolutions/{binsize}

# output of call-compartments ...
# out_prefix + "." + contact_type + ".bw"
# out_prefix + "." + contact_type + ".lam.txt"
# out_prefix + "." + contact_type + ".vecs.tsv"

already exist !
Hap1-WT-combined.hg19.EV.25kb.cis.bw
already exist !
PolII-IAA.hg19.EV.25kb.cis.bw
already exist !
PolII-NT.hg19.EV.25kb.cis.bw
already exist !
all_CTCF_combined.hg19.EV.25kb.cis.bw
already exist !
noCTCF_IAA_combined.hg19.EV.25kb.cis.bw
already exist !
CkoCT442_NT_pool.hg19.EV.25kb.cis.bw
already exist !
CkoCT442_IAA_pool.hg19.EV.25kb.cis.bw
already exist !
RAD21-IAA.hg19.EV.25kb.cis.bw
already exist !
RAD21-NT.hg19.EV.25kb.cis.bw
already exist !
CkoC442-IAA48H-R1-T1.hg19.EV.25kb.cis.bw
already exist !
CkoC442-NT-R1-T1.hg19.EV.25kb.cis.bw
already exist !
CkoCT442-AAVS1sg2-4-NT-R1-T1.hg19.EV.25kb.cis.bw
already exist !
CkoCT442-DDX55sg2-B-NT-R1-T1.hg19.EV.25kb.cis.bw
already exist !
G1-NT.hg19.EV.25kb.cis.bw
already exist !
G1-IAA.hg19.EV.25kb.cis.bw
already exist !
mutControl-NT.hg19.EV.25kb.cis.bw
already exist !
mutControl-IAA.hg19.EV.25kb.cis.bw
already exist !
mutDDX55-NT.hg19.EV.25kb.cis.bw
already exist !
mutDDX55-IAA.hg19.EV.25kb.cis.bw
already exist !
mutTAF5L-

In [44]:
# binsize = 10_000
# human_binsize = f"{int(binsize/1_000)}kb"
# # input samples here
# ev_samples = ['CkoCT442_NT_pool.hg19','CkoCT442_IAA_pool.hg19']
# clrs = []
# for s in ev_samples:
#     fname = ins_samples[s]
#     clr_uri = f"{fname}::/resolutions/{binsize}"
#     clrs.append(cooler.Cooler(clr_uri))

# # let's calculate gene coverage by pulling mRNA data from USCS
# # for our hg19 genome ...
# # this would need to be repeated for every genome, every binsize, etc
# # probably better to download mRNA data locally to avoid redownloading ...
# gene_cov_df = bioframe.frac_gene_coverage(
#                 clrs[0].bins()[:],
#                 "hg19"
#                 )
# hg19_chromsizes = bioframe.fetch_chromsizes("hg19")

# #saving to bigwig - whatever ...
# for s,_r in zip(ev_samples, ev_res):
#     # upack res, as store it:
#     eigvals, eigvecs = _r
#     bw_fname = f"{s}.EV.{human_binsize}.bw"
#     # save compartment calls as bigwig
#     bioframe.to_bigwig(
#         eigvecs,
#         hg19_chromsizes,
#         bw_fname,
#         value_field="E1"
#     )

# # running eigdecomp ...
# nproc = 1
# if nproc > 1:
#     pool = mp.Pool(nproc)
#     map_ = pool.map
# else:
#     map_ = map


# ev_res = []
# for _clr in clrs:
#     _gene_cov_df = copy(gene_cov_df)
#     _res = eigdecomp.cooler_cis_eig(
#         clr=_clr,
#         bins=_gene_cov_df,
#         regions=hg19_arms,
#         n_eigs=3,
#         phasing_track_col='coverage',
#         balance='weight',
#         ignore_diags=2,
#         clip_percentile=99.9,
#         sort_metric=None,
#         map = map_
#     )
#     ev_res.append(_res)
    
# # plotting some stuff ...
# from mpl_toolkits.axes_grid1 import make_axes_locatable

# chr1q_region = tuple(hg19_arms.iloc[1])[:-1]

# for i,_r in enumerate(ev_res):
#     a,b = _r
#     ev1 = bioframe.select(b, chr1q_region)["E1"].values

#     fig = plt.figure(figsize=(15,6),constrained_layout=True)
#     gs = fig.add_gridspec(1,3,width_ratios=[1,0.05,1])
#     axl = fig.add_subplot(gs[0,0])
#     cax = fig.add_subplot(gs[0,1])
#     axr = fig.add_subplot(gs[0,2])

#     hm = clrs[i].matrix(balance=True).fetch(chr1q_region)
#     valid_mask = np.nan_to_num(hm).sum(axis=0) == 0.
#     hm = hm[~valid_mask][:,~valid_mask]

#     ccc = axl.imshow(hm,cmap="RdYlBu_r",interpolation="nearest",norm=LogNorm())
#     plt.colorbar(ccc,cax=cax)

#     divider = make_axes_locatable(axl)
#     axev = divider.append_axes("bottom", size="15%", pad=0.08)

#     axev.plot(ev1[~valid_mask],'b-')
#     axev.axhline(0,color="grey")
#     axev.set_xlim(0,np.sum(~valid_mask))

#     # color histogram
#     _1, bin_edges, _2 = axr.hist(np.nan_to_num(np.log(hm[hm>0])).flatten(),bins=100,log=True)
#     axr.bar(bin_edges[0],np.sum(hm==0),width=-.3,align="edge",color="red",alpha=0.4,label="zero-pixels")
#     axr.set_xlabel("heatmap_signal, excluding 0-pixels")
#     axr.legend(frameon=False)